In [2]:
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px

import risk_pipeline.visualizations.line_chart as lc

In [4]:
import os
os.getcwd()

'/Users/anantdeepparihar/Overlay/overlay-risk/scripts'

# Get Data

Coinbase data (treated)

In [11]:
coin_df = pd.read_csv('risk_pipeline/outputs/data/ETH-BTC_2022-06-01-00-00_2022-10-31-00-00_600secs_treated.csv', index_col=0)

In [12]:
coin_df.head()

,time,close
0,2022-06-01 00:00:00,0.061110
1,2022-06-01 00:10:00,0.061075
2,2022-06-01 00:20:00,0.060915
3,2022-06-01 00:30:00,0.060985
4,2022-06-01 00:40:00,0.061020


In [13]:
# invert close to match uni v3
coin_df.close = 1/coin_df.close

Uniswap data

In [51]:
uni_df = pd.read_csv('../csv/WBTC-WETH-10.0mTWAP-14881677-to-15864319.csv', index_col=0)

In [52]:
uni_df.head()

,timestamp,twap
0,2022-06-01 00:01:54,16.335589
2,2022-06-01 00:11:54,16.345683
4,2022-06-01 00:21:54,16.351692
10,2022-06-01 00:31:54,16.361773
11,2022-06-01 00:41:54,16.366600


In [53]:
uni_df.reset_index(inplace=True)

# Sanity checks

In [54]:
uni_df[uni_df.twap.isna()]

,index,timestamp,twap


In [55]:
coin_df[coin_df.close.isna()]

,time,close


In [56]:
print('shape:', uni_df.shape)
print('first timestamp:', uni_df.timestamp.min())
print('last timestamp:', uni_df.timestamp.max())

shape: (21883, 3)
first timestamp: 2022-06-01 00:01:54
last timestamp: 2022-10-30 23:01:54


In [57]:
print('shape:', coin_df.shape)
print('first timestamp:', coin_df.time.min())
print('last timestamp:', coin_df.time.max())

shape: (21889, 2)
first timestamp: 2022-06-01 00:00:00
last timestamp: 2022-10-31 00:00:00


# Combine data

In [60]:
df = pd.concat([coin_df.close, uni_df.twap], axis=1)

In [63]:
df

,close,twap
0,16.363934,16.335589
1,16.373312,16.345683
2,16.416318,16.351692
3,16.397475,16.361773
4,16.388069,16.366600
...,...,...
21884,12.971010,NaN
21885,12.965124,NaN
21886,12.977743,NaN
21887,12.965964,NaN


In [65]:
print('NAs in coinbase column =', df[df.close.isna()]
print('NAs in coinbase column =', df[df.close.isna()]

,close,twap
